<a href="https://colab.research.google.com/github/fininsight/text-mining-tutorial/blob/master/9_Naive_Bayes_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes Classifier



## 구글 드라이브 파일 읽기



In [1]:
# 구글 드라이브에서 csv 파일을 읽어오기 위해 gauth 인증을 합니다.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

     |████████████████████████████████| 993kB 2.4MB/s 


In [0]:
url ='https://drive.google.com/open?id=1k-eBWXizwP75G636Ws7VrIy8VOWjKt_3'
id_spam = url.split('=')[1]

In [0]:
# it관련 기사를 판다스를 통해 읽어온다.
downloaded = drive.CreateFile({'id':id_spam}) 
downloaded.GetContentFile('9_spam.csv') 

## Naive Bayes Classifier

In [15]:
from collections import defaultdict
import pandas as pd 
import numpy as np
import math

k =0.5

input_file = pd.read_csv("9_spam.csv")
training_set = np.array(input_file)

# 범주에 속하는 문서수 세기 1(예. 긍정), 0(예. 부정))
doccnt1 = 0
doccnt0 = 0
    
# 토큰별로 문서내 빈도수 카운팅
wordfreq = defaultdict(lambda : [0, 0])
for doc, point in training_set:
      words = doc.split()
      for word in words :
        if point == 1 :
          wordfreq[word][0] += 1
        else :
          wordfreq[word][1] += 1
          
for key, (cnt1, cnt0) in wordfreq.items() :
  if cnt1 > 0 :
    doccnt1 += 1
  if cnt0 > 0 :
    doccnt0 += 1
    
#print(doccnt0)

wordprobs = defaultdict(lambda : [0, 0])
for key, (cnt1, cnt0) in wordfreq.items() :
  wordprobs[key][0] = (cnt1 + k) / (doccnt1 + 2*k)
  wordprobs[key][1] = (cnt0 + k) / (doccnt0 + 2*k)

#print(len(word_probs))

doc = "free lottery"
tokens = doc.split()

# 초기값은 모두 0으로 처리
log_prob1 = log_prob0 = 0.0

# 모든 단어에 대해 반복
for word, (prob1, prob0) in wordprobs.items():
    if word in tokens:
        log_prob1 += math.log(prob1)
        log_prob0 += math.log(prob0)
        
    else:
        log_prob1 += math.log(1.0 - prob1)
        log_prob0 += math.log(1.0 - prob0)

    
log_prob1 += math.log(doccnt1/len(wordfreq))
log_prob0 += math.log(doccnt0/len(wordfreq))
  
prob1 = math.exp(log_prob1)
prob0 = math.exp(log_prob0)
print(doc)
print("스팸확률 : {}%".format(prob1 / (prob1 + prob0)*100))
print("정상확률 : {}%".format(prob0 / (prob1 + prob0)*100))


free lottery
스팸확률 : 89.64758762435878%
정상확률 : 10.352412375641222%
